In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()
# Country Data COUNTRY table
COUNTRY_df = pd.read_sql_query("select * from COUNTRY;", conn)
# League data from LEAGUE
LEAGUE_df = pd.read_sql_query("select * from LEAGUE;", conn)
#
Team_df = pd.read_sql_query("select * from TEAM;", conn)
# Match data from Matches - just pulling 1 year for now -2015/2016 might be latest year
# You can remove restriction if you want
#MATCH_df = pd.read_sql_query("select * from MATCH where SEASON = '2015/2016';", conn)
# Match data from Matches - just pulling 1 and relevant data for now - we can remove restriction later if we want
MATCH_df = pd.read_sql_query("SELECT DATE, LEAGUE_ID, HOME_TEAM_API_ID, AWAY_TEAM_API_ID, HOME_TEAM_GOAL, AWAY_TEAM_GOAL FROM MATCH WHERE SEASON = '2015/2016';",
conn)

### We calculate the "differential" for both the home team and away team

In [2]:
# FIND MOST DOMINATE BY 

MATCH_df["HOME_DIFF"] = MATCH_df["home_team_goal"]- MATCH_df["away_team_goal"]
MATCH_df["AWAY_DIFF"] = MATCH_df["away_team_goal"]- MATCH_df["home_team_goal"]

MATCH_home_df=MATCH_df.loc[:,["date","home_team_api_id", "HOME_DIFF","league_id","away_team_api_id"]]
MATCH_away_df=MATCH_df.loc[:,["date","away_team_api_id", "AWAY_DIFF","league_id","home_team_api_id"]]

MATCH_away_df.head()
MATCH_home_df.head()



,date,home_team_api_id,HOME_DIFF,league_id,away_team_api_id
0,2015-07-24 00:00:00,9997,1,1,8342
1,2015-07-25 00:00:00,8571,1,1,9985
2,2015-07-25 00:00:00,9987,2,1,1773
3,2015-07-25 00:00:00,8573,2,1,8203
4,2015-07-25 00:00:00,10000,2,1,9994


In [3]:
Team_df

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB
5,6,8635,229.0,RSC Anderlecht,AND
6,7,9991,674.0,KAA Gent,GEN
7,8,9998,1747.0,RAEC Mons,MON
8,9,7947,NaN,FCV Dender EH,DEN
9,10,9985,232.0,Standard de Liège,STL


### We combine the home team data with the away team data to create 1 liast of games, with the differential

In [4]:
MATCH_home_df = MATCH_home_df.rename(columns={"home_team_api_id": "team_api_id","HOME_DIFF":"DIFF","away_team_api_id": "opponent" })
MATCH_away_df = MATCH_away_df.rename(columns={"away_team_api_id": "team_api_id","AWAY_DIFF":"DIFF","home_team_api_id": "opponent" })

frames = [MATCH_home_df, MATCH_away_df]
all_games_unsorted = pd.concat(frames)
all_games = all_games_unsorted.sort_values(by=["team_api_id","date"])
all_games.reset_index(inplace=True)
all_games.head() 

,index,date,team_api_id,DIFF,league_id,opponent
0,1995,2015-07-18 00:00:00,1601,-2,15722,8019
1,2084,2015-07-24 00:00:00,1601,2,15722,8028
2,2173,2015-08-03 00:00:00,1601,1,15722,8245
3,2188,2015-08-10 00:00:00,1601,0,15722,8033
4,2197,2015-08-16 00:00:00,1601,-1,15722,1957


In [5]:
### We want to figure out the what the differential was for the next game as well.  
### We will make a copy of the table, and add "ng_" as a suffix to the columns for "next game"

In [6]:
next_game = all_games.copy()
next_game = next_game.rename(columns={"DIFF": "ng_DIFF",
                                      "team_api_id": "ng_team_api_id2",
                                      "league_id":   "ng_league_id",
                                      "date":   "ng_date"
                                                                     })

next_game.head()

,index,ng_date,ng_team_api_id2,ng_DIFF,ng_league_id,opponent
0,1995,2015-07-18 00:00:00,1601,-2,15722,8019
1,2084,2015-07-24 00:00:00,1601,2,15722,8028
2,2173,2015-08-03 00:00:00,1601,1,15722,8245
3,2188,2015-08-10 00:00:00,1601,0,15722,8033
4,2197,2015-08-16 00:00:00,1601,-1,15722,1957


### We will merge the original data to the copy, but shift the copy data by one row, so it will line up with the next game

In [7]:
#df_complete = pd.merge(all_games, next_game.shift(-1).fillna(0).astype(int), left_index=True, right_index=True)
df_complete = pd.merge(all_games, next_game.shift(-1).fillna(0), left_index=True, right_index=True)
df_complete.head(50)

,index_x,date,team_api_id,DIFF,league_id,opponent_x,index_y,ng_date,ng_team_api_id2,ng_DIFF,ng_league_id,opponent_y
0,1995,2015-07-18 00:00:00,1601,-2,15722,8019,2084.0,2015-07-24 00:00:00,1601.0,2.0,15722.0,8028.0
1,2084,2015-07-24 00:00:00,1601,2,15722,8028,2173.0,2015-08-03 00:00:00,1601.0,1.0,15722.0,8245.0
2,2173,2015-08-03 00:00:00,1601,1,15722,8245,2188.0,2015-08-10 00:00:00,1601.0,0.0,15722.0,8033.0
3,2188,2015-08-10 00:00:00,1601,0,15722,8033,2197.0,2015-08-16 00:00:00,1601.0,-1.0,15722.0,1957.0
4,2197,2015-08-16 00:00:00,1601,-1,15722,1957,2204.0,2015-08-22 00:00:00,1601.0,3.0,15722.0,177361.0
5,2204,2015-08-22 00:00:00,1601,3,15722,177361,2213.0,2015-08-29 00:00:00,1601.0,-2.0,15722.0,8021.0
6,2213,2015-08-29 00:00:00,1601,-2,15722,8021,2220.0,2015-09-13 00:00:00,1601.0,1.0,15722.0,8020.0
7,2220,2015-09-13 00:00:00,1601,1,15722,8020,2231.0,2015-09-20 00:00:00,1601.0,-3.0,15722.0,8673.0
8,2231,2015-09-20 00:00:00,1601,-3,15722,8673,2005.0,2015-09-26 00:00:00,1601.0,-1.0,15722.0,2186.0
9,2005,2015-09-26 00:00:00,1601,-1,15722,2186,2014.0,2015-10-02 00:00:00,1601.0,-2.0,15722.0,8023.0


### Oops!  For the last game of the season (where we don't have a next game), checking the next game would have returned the next team's first game.  We should just zero that out.      

In [8]:
#df.loc[df.First_name == 'Bill', 'name_match'] = 'Match' 
df_complete.sort_values(by=["team_api_id","date"]).head()
df_complete.loc[df_complete.team_api_id != df_complete.ng_team_api_id2, "ng_DIFF"] = 0
df_complete.head(2)

,index_x,date,team_api_id,DIFF,league_id,opponent_x,index_y,ng_date,ng_team_api_id2,ng_DIFF,ng_league_id,opponent_y
0,1995,2015-07-18 00:00:00,1601,-2,15722,8019,2084.0,2015-07-24 00:00:00,1601.0,2.0,15722.0,8028.0
1,2084,2015-07-24 00:00:00,1601,2,15722,8028,2173.0,2015-08-03 00:00:00,1601.0,1.0,15722.0,8245.0


### Let's delete all the extra columns we created

In [9]:
df_complete.drop(['index_x','index_y','ng_date','ng_league_id'], axis=1, inplace=True)
df_complete.head()

# df_complete rules!  

,date,team_api_id,DIFF,league_id,opponent_x,ng_team_api_id2,ng_DIFF,opponent_y
0,2015-07-18 00:00:00,1601,-2,15722,8019,1601.0,2.0,8028.0
1,2015-07-24 00:00:00,1601,2,15722,8028,1601.0,1.0,8245.0
2,2015-08-03 00:00:00,1601,1,15722,8245,1601.0,0.0,8033.0
3,2015-08-10 00:00:00,1601,0,15722,8033,1601.0,-1.0,1957.0
4,2015-08-16 00:00:00,1601,-1,15722,1957,1601.0,3.0,177361.0


### Now, lets find the most dominant team.  We will do that by grouping by the bext team


In [10]:
df_teams = df_complete.groupby(['league_id','team_api_id'], as_index=False).sum()
df_teams.head(50)

# df_teams = df_complete.groupby(['league_id','team_api_id']).sum()
# df_teams.head(50)



,league_id,team_api_id,DIFF,opponent_x,ng_team_api_id2,ng_DIFF,opponent_y
0,1,1773,-11,810642,53374.0,-9.0,808900.0
1,1,8203,-2,797782,246113.0,0.0,797387.0
2,1,8342,34,797504,250266.0,35.0,945592.0
3,1,8475,-17,797238,254260.0,-16.0,798541.0
4,1,8571,-4,797046,257132.0,-5.0,795264.0
5,1,8573,11,797042,257198.0,9.0,798749.0
6,1,8635,22,796918,259051.0,21.0,796967.0
7,1,9985,-10,794218,299551.0,-9.0,1060228.0
8,1,9986,-3,794216,299581.0,-4.0,521408.0
9,1,9987,12,794214,299614.0,10.0,802442.0


### For each league, we will append the team that has the maximum Differential.  These are our dominant teams

In [11]:
leagues = df_teams.league_id.unique()
x = []
for l in leagues:
    df_test = df_teams[df_teams.league_id == l]
    x.append(df_test['team_api_id'].loc[df_test['DIFF'].idxmax()])
print(x)

[8342, 8586, 9847, 9823, 9885, 8593, 8673, 9772, 9925, 8634, 9931]


In [12]:
df_complete.opponent_x.unique()

array([  8019,   8028,   8245,   8033,   1957, 177361,   8021,   8020,
         8673,   2186,   8023,   2182,   8025,  10265,   8030,   9987,
         8635,  10001,   9986,   8203,   9997,   8571,  10000,   9985,
       274581,   9994,   8475,   8342,   9991,   8573,   1601,  10212,
        10214,   7844,   8348, 158085,   7842,   9807,  10264,  10238,
         9773, 188163,   6403,   9772,   9768,  10215,   8613,   7841,
        10242,   9847,   8121,   8639,   9829,   9747,   9851,   7819,
         9941,   9853,   9831,   9827,   9837,   7794,   8689,  10249,
         9830,   8592,   9748,   6367,   7788,   8526,   8611,  10228,
        10218,  10217,  10235,   8525,   8277,   9908,   8674,   9791,
         9803,   8593,   8464,  10229,   8640,   6413,   6391,   8372,
         8305,  10205,   8633,   8560,  10267,   9783,   9869,   8603,
         8558,   8370,   8315,   9864,   9906,   8581,   9910,   8306,
         8302,   8634,  10190,  10191,   9824,   9956,  10192,   9931,
      

In [24]:
# How to show dominant team in a visual?

#good_movies = movie_file_df.loc[movie_file_df["IMDB"] > 7, [ "FILM", "IMDB", "IMDB_user_vote_count"]]
 
dom_match = df_complete[df_complete.opponent_x.isin(x)]
dom_match.head()
# opponent_x is the dom team
# opponent_Y is the next game's opponent

# merge with df_teams to get season DIFF for: team's season diff, opp x season diff, opp y season diff

# bar chart of dominant team  in each league Team season diff / # games



dom_match.opponent_y.value_counts()


AttributeError: 'function' object has no attribute 'sum'

In [22]:
df_teams.head()

df_DIFF = df_teams[["team_api_id","DIFF"]]

df_DIFF = df_DIFF.rename(columns={"DIFF": "total_DIFF"})


df_DIFF.head(5)
    
#     df_complete.drop(['index_x','index_y','ng_date','ng_league_id'], axis=1, inplace=True)

# next_game = next_game.rename(columns={"DIFF": "ng_DIFF",
#                                       "team_api_id": "ng_team_api_id2",
#                                       "league_id":   "ng_league_id",
#                                       "date":   "ng_date"
#                                                                      })

,team_api_id,total_DIFF
0,1773,-11
1,8203,-2
2,8342,34
3,8475,-17
4,8571,-4


In [15]:
# set team_api_id's DIFF
merge_table = pd.merge(dom_match, df_DIFF,left_on="team_api_id", right_on="team_api_id")
 
merge_table = merge_table.rename(columns={"opponent_x": "dom_team",
                                          "team_api_id": "team",
                                      "opponent_y": "ng_opponent",
                                      "total_DIFF":   "team_season_DIFF"                                  
                                                                     })


In [16]:
merge_table = pd.merge(merge_table, df_DIFF,left_on="dom_team", right_on="team_api_id")
merge_table.head(5)


,date,team,DIFF,league_id,dom_team,ng_team_api_id2,ng_DIFF,ng_opponent,team_season_DIFF,team_api_id,total_DIFF
0,2015-09-20 00:00:00,1601,-3,15722,8673,1601.0,-1.0,2186.0,-9,8673,30
1,2016-02-28 00:00:00,1601,-2,15722,8673,1601.0,-1.0,2186.0,-9,8673,30
2,2015-08-30 00:00:00,1957,0,15722,8673,1957.0,-2.0,8025.0,-17,8673,30
3,2016-02-14 00:00:00,1957,-4,15722,8673,1957.0,1.0,8025.0,-17,8673,30
4,2015-10-25 00:00:00,2182,1,15722,8673,2182.0,0.0,8025.0,-1,8673,30


In [17]:
merge_table = merge_table.rename(columns={"total_DIFF": "dom_season_DIFF"})
merge_table.drop(['team_api_id'], axis=1, inplace=True)


In [18]:
merge_table.head()

,date,team,DIFF,league_id,dom_team,ng_team_api_id2,ng_DIFF,ng_opponent,team_season_DIFF,dom_season_DIFF
0,2015-09-20 00:00:00,1601,-3,15722,8673,1601.0,-1.0,2186.0,-9,30
1,2016-02-28 00:00:00,1601,-2,15722,8673,1601.0,-1.0,2186.0,-9,30
2,2015-08-30 00:00:00,1957,0,15722,8673,1957.0,-2.0,8025.0,-17,30
3,2016-02-14 00:00:00,1957,-4,15722,8673,1957.0,1.0,8025.0,-17,30
4,2015-10-25 00:00:00,2182,1,15722,8673,2182.0,0.0,8025.0,-1,30


In [19]:
merge_table = pd.merge(merge_table, df_DIFF,left_on="ng_opponent", right_on="team_api_id")
merge_table = merge_table.rename(columns={"total_DIFF": "ng_opp_season_DIFF"})
merge_table.head(5)


,date,team,DIFF,league_id,dom_team,ng_team_api_id2,ng_DIFF,ng_opponent,team_season_DIFF,dom_season_DIFF,team_api_id,ng_opp_season_DIFF
0,2015-09-20 00:00:00,1601,-3,15722,8673,1601.0,-1.0,2186.0,-9,30,2186,15
1,2016-02-28 00:00:00,1601,-2,15722,8673,1601.0,-1.0,2186.0,-9,30,2186,15
2,2015-09-11 00:00:00,8021,0,15722,8673,8021.0,2.0,2186.0,4,30,2186,15
3,2016-02-21 00:00:00,8021,-1,15722,8673,8021.0,1.0,2186.0,4,30,2186,15
4,2015-07-26 00:00:00,8033,-5,15722,8673,8033.0,-1.0,2186.0,-9,30,2186,15


In [20]:
merge_table.drop(['team_api_id'], axis=1, inplace=True)

In [21]:
#merge_table.value_count("ng_opponent")
merge_table.ng_opponent.value_counts()


8372.0      32
8348.0      12
8023.0       8
8540.0       8
8178.0       8
8429.0       7
8262.0       7
10243.0      6
8543.0       6
9860.0       6
10252.0      6
2186.0       6
10218.0      6
274581.0     6
10191.0      6
9991.0       6
8678.0       5
8456.0       5
10229.0      5
9829.0       5
9807.0       5
9994.0       5
10190.0      5
1773.0       4
9904.0       4
8025.0       4
9905.0       4
8030.0       4
8573.0       4
188163.0     4
            ..
8524.0       2
6413.0       2
9831.0       2
10194.0      2
8533.0       2
8535.0       2
7842.0       2
9938.0       2
8245.0       1
8455.0       1
8654.0       1
8611.0       1
9817.0       1
8472.0       1
8639.0       1
9791.0       1
9803.0       1
10242.0      1
8475.0       1
10261.0      1
9768.0       1
8673.0       1
208931.0     1
8674.0       1
10264.0      1
9850.0       1
10003.0      1
8603.0       1
8234.0       1
8066.0       1
Name: ng_opponent, Length: 117, dtype: int64